In [ ]:
import re
import json
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-audio", "facebook/musicgen-large", device=1)

In [ ]:

prompt = "acapella castrati chorus"
music = synthesiser(prompt, forward_params={"do_sample": True})

scipy.io.wavfile.write(f"{prompt.replace(' ', '_')}.wav", rate=music["sampling_rate"], data=music["audio"])

In [ ]:
checkpoint = 100000
dev_src = "data/raw/recipes/dev_ar.src"
dev_tgt = "data/raw/recipes/dev_ar.tgt"
run = "gpt2_lr5e-05"
model_path = f"my_output/recipes/{run}/model/checkpoint-{checkpoint}"



In [ ]:
def load_from_lines(path):
    with open(path, 'r') as f:
        return [line.strip() for line in f]
dev = {
    'src': load_from_lines(dev_src),
    'tgt': load_from_lines(dev_tgt)
}


In [ ]:


tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
model.cuda()

In [ ]:
sample = dev['src'][0]
n_return = 10
# tokenize input
input_ids = tokenizer.encode(sample, return_tensors='pt')
# generate text until the output length (which includes the context length) reaches 50
outputs = model.generate(
    input_ids.cuda(), 
    max_length=512+128,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=n_return,
    eos_token_id=tokenizer.eos_token_id
)
# take out the input_ids from the output and decode it
for i in range(n_return):     
    output = outputs[i, len(input_ids[0]):].tolist()
    print(tokenizer.decode(output, skip_special_tokens=True))
    print('-------------------')